# Importar librerías

In [1]:
import kagglehub
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

# Importar datos

In [2]:
# Url del dataset en Kaggle: https://www.kaggle.com/datasets/kundanbedmutha/student-performance-dataset/data

# Descargar el dataset de Kaggle
path = kagglehub.dataset_download("kundanbedmutha/student-performance-dataset")

# Listar los archivos en el directorio descargado
file_list = os.listdir(path)

# Seleccionar el primer archivo CSV encontrado
csv_file_name = file_list[0]

# Construir la ruta completa al archivo CSV
file_path = os.path.join(path, csv_file_name)

# Cargar el archivo CSV en un DataFrame de pandas
df = pd.read_csv(file_path)

# Mostrar las primeras 5 filas del DataFrame
print("Contenido del DataFrame:")
df.keys()

100%|██████████| 516k/516k [00:00<00:00, 1.48MB/s]

Extracting files...
Contenido del DataFrame:


Index(['student_id', 'age', 'gender', 'school_type', 'parent_education',
       'study_hours', 'attendance_percentage', 'internet_access',
       'travel_time', 'extra_activities', 'study_method', 'math_score',
       'science_score', 'english_score', 'overall_score', 'final_grade'],
      dtype='object')

In [3]:
X, y = df.overall_score, df.final_grade
X

0        53.1
1        61.3
2        89.6
3        41.6
4        25.4
         ... 
24995    46.1
24996    56.5
24997    36.7
24998    34.1
24999    31.4
Name: overall_score, Length: 25000, dtype: float64

In [4]:
X.shape

(25000,)

In [5]:
y

0        e
1        d
2        b
3        e
4        f
        ..
24995    e
24996    d
24997    f
24998    f
24999    f
Name: final_grade, Length: 25000, dtype: object

In [6]:
y.shape

(25000,)

# Separar datos de Entranemiento y Prueba

In [7]:
# Definir una semilla para la reproducibilidad
random_seed = 42

# Crear el clasificador SGD y establecer la semilla
classifier = SGDClassifier(random_state=random_seed)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

# Entrenar Clasificador Binario 

In [ ]:
# Convertir X_train y X_test a 2D
X_train_2d = X_train.values.reshape(-1, 1)
X_test_2d = X_test.values.reshape(-1, 1)

# Definir manualmente las etiquetas binarias para 'A' y 'No A'
y_train_manual = (y_train.str.lower() == 'a').astype(int)
y_test_manual  = (y_test.str.lower() == 'a').astype(int)

y_train_manual.value_counts()

# Entrenar el clasificador
#.fit(X_train_2d, y_train_manual)

SGDClassifier(random_state=42)

In [9]:
# Realizar una predicción con un nuevo valor
new_sample = [[15]]  # forma 2D
pred = classifier.predict(new_sample)
print(pred)

[0]


# Medidas de rendimiento

In [10]:
cross_val_score(classifier, X_train_2d, y_train_manual, cv=3, scoring="accuracy")

array([0.98425079, 0.98845058, 0.9849985 ])

In [11]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3, shuffle=True, random_state=random_seed)

for train_index, test_index in skfolds.split(X_train_2d, y_train_manual):
    clone_clf = clone(classifier)
    X_train_folds = X_train_2d[train_index]
    y_train_folds = y_train_manual.iloc[train_index]
    X_test_fold = X_train_2d[test_index]
    y_test_fold = y_train_manual.iloc[test_index]

    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))

0.9526023698815059
0.9938503074846258
0.9893489348934893


In [12]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier()
dummy_clf.fit(X_train_2d, y_train_manual)
print(any(dummy_clf.predict(X_test_2d)))

False


In [13]:
cross_val_score(dummy_clf, X_train_2d, y_train_manual, cv=3, scoring="accuracy")

array([0.95260237, 0.95245238, 0.95259526])

Matriz de Confusión

In [14]:
from sklearn.model_selection import cross_val_predict

# Obtener las predicciones de entrenamiento mediante validación cruzada
y_train_pred = cross_val_predict(classifier, X_train_2d, y_train_manual, cv=3)


In [15]:
from sklearn.metrics import confusion_matrix

# Calcular la matriz de confusión
conf_mx = confusion_matrix(y_train_manual, y_train_pred)

print("Matriz de confusión:\n", conf_mx)
print("\nNo 'A' predichas como no 'A':", conf_mx[0, 0])
print("No 'A' predichas como 'A':", conf_mx[0, 1])
print("'A' predichas como no 'A':", conf_mx[1, 0])
print("'A' predichas como 'A':", conf_mx[1, 1])

Matriz de confusión:
 [[19051     0]
 [  282   667]]

No 'A' predichas como no 'A': 19051
No 'A' predichas como 'A': 0
'A' predichas como no 'A': 282
'A' predichas como 'A': 667


In [16]:
# Matriz de confusión perfecta
y_train_perfect_predictions = y_train_manual.copy()

conf_mx_perfect = confusion_matrix(y_train_manual, y_train_perfect_predictions)

print("Matriz de confusión perfecta:\n", conf_mx_perfect)
print("\nNo 'A' predichas como no 'A':", conf_mx_perfect[0, 0])
print("No 'A' predichas como 'A' :", conf_mx_perfect[0, 1])
print("'A' predichas como no 'A' :", conf_mx_perfect[1, 0])
print("'A' predichas como 'A' :", conf_mx_perfect[1, 1])

Matriz de confusión perfecta:
 [[19051     0]
 [    0   949]]

No 'A' predichas como no 'A': 19051
No 'A' predichas como 'A' : 0
'A' predichas como no 'A' : 0
'A' predichas como 'A' : 949
